# Reproduce DL
## Automated Pavement Crack Segmentation

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary


class ResBlock(nn.Module):
    """
    Create a Residual block
    """

    def __init__(self, in_channels, out_channels, stride):
        super(ResBlock, self).__init__()
        self.stride_one = (stride == 1)

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv_shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        self.bn_shortcut = nn.BatchNorm2d(out_channels)

        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.relu(y)

        y = self.conv2(y)
        y = self.bn2(y)

        if not self.stride_one:
            x = self.bn_shortcut(self.conv_shortcut(x))
        y += x

        return self.relu(y)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3 input image channels, 64 output channels, 7x7 convolution, stride 2
        # -- Blue --
        self.conv1 = nn.Conv2d(3, 64, 7, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        # -- Green --
        # maxpooling stride default is kernel size
        self.maxpool_1_to_rs1 = nn.MaxPool2d(3, stride=2)

        # Residual block 1
        self.conv_rs1_1 = ResBlock(64, 64, 1)
        self.conv_rs1_2 = ResBlock(64, 64, 1)
        self.conv_rs1_3 = ResBlock(64, 64, 1)

        # Residual block 2
        self.conv_rs2_1 = ResBlock(64, 128, 2)
        self.conv_rs2_2 = ResBlock(128, 128, 1)
        self.conv_rs2_3 = ResBlock(128, 128, 1)
        self.conv_rs2_4 = ResBlock(128, 128, 1)

        # Residual block 3
        self.conv_rs3_1 = ResBlock(128, 256, 2)
        self.conv_rs3_2 = ResBlock(256, 256, 1)
        self.conv_rs3_3 = ResBlock(256, 256, 1)
        self.conv_rs3_4 = ResBlock(256, 256, 1)
        self.conv_rs3_5 = ResBlock(256, 256, 1)
        self.conv_rs3_6 = ResBlock(256, 256, 1)

        # Residual block 4
        self.conv_rs4_1 = ResBlock(256, 512, 2)
        self.conv_rs4_2 = ResBlock(512, 512, 1)
        self.conv_rs4_3 = ResBlock(512, 512, 1)

        # -- Yellow --
        # 64/128 input image channels, 128 output channels, 1x1 convolution, stride 1
        # Not sure if stride should be 1...
        # --
        # Green (residual) block to Yellow block
        self.conv_gr_to_yel_1 = nn.Conv2d(64, 128, 1, stride=1)
        self.conv_gr_to_yel_2 = nn.Conv2d(64, 128, 1, stride=1)
        self.conv_gr_to_yel_3 = nn.Conv2d(128, 128, 1, stride=1)
        self.conv_gr_to_yel_4 = nn.Conv2d(128, 128, 1, stride=1)
        # Green (residual) block to Purple block
        self.conv_gr_to_purp = nn.Conv2d(512, 512, 1, stride=1)

    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.relu1(y)
        # y_y1 = self.conv_gr_to_yel_1(y)

        y = self.maxpool_1_to_rs1(y)
        y = self.conv_rs1_1(y)
        y = self.conv_rs1_2(y)
        y = self.conv_rs1_3(y)
        # y_y2 = self.conv_gr_to_yel_2(y)

        y = self.conv_rs2_1(y)
        y = self.conv_rs2_2(y)
        y = self.conv_rs2_3(y)
        y = self.conv_rs2_4(y)
        # y_y3 = self.conv_gr_to_yel_3(y)

        y = self.conv_rs3_1(y)
        y = self.conv_rs3_2(y)
        y = self.conv_rs3_3(y)
        y = self.conv_rs3_4(y)
        y = self.conv_rs3_5(y)
        y = self.conv_rs3_6(y)
        # y_y4 = self.conv_gr_to_yel_4(y)

        y = self.conv_rs4_1(y)
        y = self.conv_rs4_2(y)
        y = self.conv_rs4_3(y)
        # y_y5 = self.conv_gr_to_purp(y)

        return y


net = Net()
# print(net)
summary(net, input_size=(3, 320, 480))

/root/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 157, 237]           9,472
       BatchNorm2d-2         [-1, 64, 157, 237]             128
              ReLU-3         [-1, 64, 157, 237]               0
         MaxPool2d-4          [-1, 64, 78, 118]               0
            Conv2d-5          [-1, 64, 78, 118]          36,928
       BatchNorm2d-6          [-1, 64, 78, 118]             128
              ReLU-7          [-1, 64, 78, 118]               0
            Conv2d-8          [-1, 64, 78, 118]          36,

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=411d58e9-cb4b-4924-bef0-2f383eff0187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>